<a href="https://colab.research.google.com/github/naveedkhalid091/Learn_Agentic_AI/blob/main/step02_generative_ai_for_beginners/02(b)_updated_RAG_implementation_with_PineconeDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Implementation of RAG projects:**

For RAG projects in langchain, you need to store and retreive your data.

You need the **following environment** to set in your project.

1. Install the langchain in your project for creating flexibility in switching the chat models.
2. Firstly, you need a database for data storage and its access key.  
3. An Embedding model for vectorization of your data.
4. LLM model for conversations and its access key.

Lets Install the above environment first.

In [35]:
!pip install -U -q langchain

In [36]:
!pip install -U -q langchain-pinecone

In [37]:
!pip install -U -q langchain_google_genai

## **Gettting Access of `PINECONE` & `GEMINI` using API Keys:**

In [38]:
from google.colab import userdata
import os
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY') # Getting access of PINECONE Database
os.environ['GOOGLE_API_KEY']=userdata.get('GOOGLE_API_KEY') # Getting access of Gemini

## **Initialization of Pinecone client:**

Initializing the `Pinecone client (pc)` is the important step as this client allows you to perform various operations such as creating indexes, inserting vectors, and executing queries.

In [39]:
from pinecone import Pinecone
pc=Pinecone()

## **Create an Index in PINECONE using above client**.

**You can optionally check the existing index name using be below code to prevent duplicates:**

* **i)** First check if the index already exist with the same choosen name.

* **ii)** Secondly create an index if the same name index is not already created.

In [40]:
# i) Checking the index name if it is already exist?

existing_indexes=[]

checking_db_indexes=pc.list_indexes()
print(existing_indexes)

for info_index in checking_db_indexes:
  existing_indexes.append(info_index.name)

print(existing_indexes)

[]
['online-rag-project', 'my-9th-chem-book', 'second-rag-project', 'family-structure']


In [41]:
# ii) Creation of Index
from pinecone import ServerlessSpec


index_name="my-books"


if index_name in existing_indexes:
  print(f"Index {index_name} already exist")
else:
  # PROCEED WITH INDEX CREATION
  pc.create_index(
    name=index_name,
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

## **Accessing the above created Index:**

- Accessing the index will help us inserting the vectors/embedded data through the below line.

In [43]:
index=pc.Index(index_name)

**Note:** PINECONE database setup is successfully completed. Now you need to setup embedding model for vectorization and chunking of your data.

You can also varify your created index into the PINECONE database by signing into your database and navigate to **`Database->Indexes`**.   

## **Select Embedding model:**

This model will first ensure that all of your data has been vectorized (converted into numbers) and ready for entering into the Pinecone database through above **`index`** variable.

The Embedding model can be selected/imported from the `langchain_google_genai` library as below:   

In [44]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding_model=GoogleGenerativeAIEmbeddings(model="models/embedding-001")

**At this stage, the database is setup and embedding model is also selected for the vectorization of data, finally the vecotrized data will enter into the Pinecone database**

The data that need to be vectorized consist of either simple `text`, `small file` or a `large file`.

The **`simple text`** & **`small file`** will not be chunked but the **`large files`** will first went through the chunking process and then after chunking, the vectorization will be done.  

Lets run all the possiblities one by one.

## **Import PineconeVectorStore**:

- The **`PineconeVectorStore`** is a class of the LangChain framework that not only **embed your files automatically** before storing the files into vector databases but it also simplifies the process of `storing` and `retrieving` vector embeddings (the text of your file).  

- However, you can **convert text into vectors** manually through the `embed_query` method as follow:

 `vector_text=embedding_model.embed_query("Hello, I am Naveed")`
  `print(vector_text)`

But This manual effort has been eliminated by the vector store:






In [45]:
from langchain_pinecone import PineconeVectorStore

## create a vector store client
vector_store=PineconeVectorStore(index=index, embedding=embedding_model)

**While:**
- The `index` parameter tells the vector store where to store and retrieve the vector embeddings.
- The embedding parameter defines how the textual data is converted into vectors.

## 1. **Prepare Documents for the upload**

 - Import the Document from `langchain_core.documents`.
 - Create a `Document` Object which contains the link of text/file you wanted to store into the Database.
 - Rather then writting the manual IDs for each document, you can import the `uuid` library for generating the random and unique IDs for each document.

The relevent coding these steps is given below:    

In [46]:
from langchain_core.documents import Document

document_1=Document(
    page_content="Chemistry book",
    metadata={"/content/Chemistry 10.pdf":"Chemistry Book"} # path of the file and its title in dictionary
)


# put all the ducments into an array because only array is accepted while adding the documents in below step
documents=[document_1]

In [47]:
len(documents)

1

## Add document into Database after importing `uuid` library

In [48]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)


['5f77ac62-1334-4d1f-babb-2b1ed38fdb43']

## **Querying from LLM regarding the document uploaded to the vector database**:

We cannot directly ask the large language model (LLM) to get information from the vector database. Instead, we follow these steps:
 - 1. **Fetch Relevant Data from database:** First, we use a function called similarity_search to find the most relevant data from the vector database based on the user's query.
 - 2. **Combine Query and Data:** Next, we create a function that combines the user's query with the relevant data fetched from the database. This helps the LLM understand what information is needed.
 - 3. **Send to LLM:** Finally, we send the combined data (the query and the relevant database information) to the LLM. The LLM will then give a response based on this combined information.  

This approach ensures that the LLM can respond accurately, using both the query and the relevant data retrieved from the database.

In [55]:
## similarity search

query= "Book"

db_result=vector_store.similarity_search(
    query,
    k=10,
    )

for res in db_result:
  print(f"content:{res.page_content}\nMetadata:{res.metadata}\n")

content:Chemistry book
Metadata:{'/content/Chemistry 10.pdf': 'Chemistry Book'}



In [56]:
# llm calling

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.9,
    max_tokens=1000,
    # other params...
)

## Define a function for final answer from LLM:

In [57]:
# define a function
def answer_to_user(query:str):
  # Vector Search
  vector_results = vector_store.similarity_search(query, k=2)
  ## invoking llm
  final_answer = llm.invoke (f'''Answer this user Query: {query}, Here are some ref to answer {vector_results}, ''')
  return final_answer

In [59]:
# calling function with query

answer=answer_to_user(" how many chepters are avaialble in chemistry book")
print(answer.content)

1
The provided text only mentions the existence of a "Chemistry book" in a PDF file.  There is no information within the provided context about the number of chapters it contains.  More information from the actual PDF is needed to answer the question.
